In [11]:
import os
from PIL import Image
from itertools import product
import json

In [8]:
images_dir = "/home/tim/projekte/bar_website/streamlit-dungeon/graphics/bar_tilesets"
splitted_images_dir = "/home/tim/projekte/bar_website/streamlit-dungeon/graphics/splitted_images"

In [ ]:
def split_img_into_tiles(filename, dir_in, dir_out, d):
    name, ext = os.path.splitext(filename)
    img = Image.open(os.path.join(dir_in, filename))
    w, h = img.size
    
    grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
    for i, j in grid:
        box = (j, i, j+d, i+d)
        out = os.path.join(dir_out, f'{name}_{i}_{j}{ext}')
        img.crop(box).save(out)

In [ ]:
for image in os.listdir(images_dir):
    split_img_into_tiles(
        image,
        images_dir,
        splitted_images_dir,
        16
    )

In [ ]:
mapping = {"0": "black.png"}
walkable_mapping = {"0": True}

row = -1
col = 1
tilemap_width = 32
for image in os.listdir(splitted_images_dir):
    if image.split("_")[-1] == "0.png":
        row += 1
        col = 1
    tile_id = col+(row*tilemap_width)
    mapping[str(tile_id)] = image
    walkable_mapping[int(tile_id)] = True
    col += 1

In [87]:
with open("graphics/image_mapping.json", "w") as f:
    json.dump(mapping, f)

In [151]:
with open("graphics/walkable_mapping.json", "w") as f:
    json.dump(walkable_mapping, f)

In [75]:
with open("graphics/image_mapping.json", "r") as f:    
    data = json.load(f)
print(data)

{'1': 'Bar00_TopDownHouse_DoorsAndWindows_0_0.png', '2': 'Bar00_TopDownHouse_DoorsAndWindows_0_16.png', '3': 'Bar00_TopDownHouse_DoorsAndWindows_0_32.png', '4': 'Bar00_TopDownHouse_DoorsAndWindows_0_48.png', '5': 'Bar00_TopDownHouse_DoorsAndWindows_0_64.png', '6': 'Bar00_TopDownHouse_DoorsAndWindows_0_80.png', '7': 'Bar00_TopDownHouse_DoorsAndWindows_0_96.png', '8': 'Bar00_TopDownHouse_DoorsAndWindows_0_112.png', '9': 'Bar00_TopDownHouse_DoorsAndWindows_0_128.png', '10': 'Bar00_TopDownHouse_DoorsAndWindows_0_144.png', '11': 'Bar00_TopDownHouse_DoorsAndWindows_0_160.png', '12': 'Bar00_TopDownHouse_DoorsAndWindows_0_176.png', '13': 'Bar00_TopDownHouse_DoorsAndWindows_0_192.png', '14': 'Bar00_TopDownHouse_DoorsAndWindows_0_208.png', '15': 'Bar00_TopDownHouse_DoorsAndWindows_0_224.png', '16': 'Bar00_TopDownHouse_DoorsAndWindows_0_240.png', '17': 'Bar00_TopDownHouse_DoorsAndWindows_0_256.png', '18': 'Bar00_TopDownHouse_DoorsAndWindows_0_272.png', '33': 'Bar00_TopDownHouse_DoorsAndWindows_16

---

In [15]:
import pandas as pd

In [39]:
width = pd.read_json("graphics/testmap.json").layers[0]['chunks'][0]['width']

In [37]:
data = pd.read_json("graphics/testmap.json").layers[0]['chunks'][0]['data']

In [50]:
pd.DataFrame([data[x:x+width] for x in range(0, len(data), width)])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,92,93,94,93,94,93,94,93,94,93,94,93,94,93,94,95
1,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113
2,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113
3,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113
4,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113
5,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113
6,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113
7,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113
8,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113
9,110,111,112,111,112,111,112,111,112,111,112,111,112,111,112,113


In [ ]:
import json



In [139]:

def get_nested_lists_from_json(json_file):
    with open(f"graphics/{json_file}") as json_data:
        data = json.load(json_data)

    layer_data = []
    for layer in data['layers']:
        layer_width = layer['width']
        layer_height = layer['height']
        chunks = layer['chunks']
        df = pd.DataFrame(index=range(layer_height), columns=range(layer_width))
        for chunk in chunks:
            chunk_width = chunk['width']
            chunk_height = chunk['height']
            data = chunk['data']
            x_offset = chunk['x']
            y_offset = chunk['y']

            for i in range(chunk_height):
                for j in range(chunk_width):
                    df.iloc[y_offset+i, x_offset+j] = data[i*chunk_width+j]
        layer_data.append(df.values)
    return layer_data

In [140]:
not_walkable_tiles = get_nested_lists_from_json("not_walkable_set.json")

In [146]:
not_walkable_tiles

[array([[103, 104, 105, ..., 0, 0, 0],
        [135, 136, 137, ..., 0, 0, 0],
        [263, 264, 265, ..., 0, 0, 0],
        ...,
        [1217, 1218, 1219, ..., 0, 0, 0],
        [1249, 1250, 1251, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=object)]

In [147]:
for layer in not_walkable_tiles:
    for row in layer:
        for tile in row:
            if tile != 0:
                walkable_mapping[tile] = False


In [150]:
walkable_mapping

{'0': True,
 1: True,
 2: True,
 3: True,
 4: True,
 5: True,
 6: True,
 7: True,
 8: True,
 9: False,
 10: False,
 11: False,
 12: False,
 13: True,
 14: True,
 15: False,
 16: False,
 17: False,
 18: False,
 33: False,
 34: False,
 35: True,
 36: True,
 37: True,
 38: True,
 39: True,
 40: True,
 41: False,
 42: False,
 43: False,
 44: False,
 45: True,
 46: True,
 47: False,
 48: False,
 49: False,
 50: False,
 65: False,
 66: False,
 67: False,
 68: False,
 69: False,
 70: True,
 71: True,
 72: True,
 73: True,
 74: True,
 75: True,
 76: True,
 77: True,
 78: True,
 79: True,
 80: True,
 81: True,
 82: True,
 97: False,
 98: False,
 99: False,
 100: False,
 101: False,
 102: True,
 103: False,
 104: False,
 105: False,
 106: False,
 107: True,
 108: True,
 109: False,
 110: False,
 111: False,
 112: False,
 113: True,
 114: True,
 129: True,
 130: True,
 131: False,
 132: False,
 133: False,
 134: True,
 135: False,
 136: False,
 137: False,
 138: False,
 139: False,
 140: False,
 

In [ ]:
for image_id in mapping.keys():
    walkable 

In [ ]:
for layer in layer_data:
    for row in layer:
        for tile in row:
            if tile != 0:
                print(mapping[str(tile)])

Bar00_TopDownHouse_DoorsAndWindows_48_96.png
Bar00_TopDownHouse_DoorsAndWindows_48_112.png
Bar00_TopDownHouse_DoorsAndWindows_48_128.png
Bar00_TopDownHouse_DoorsAndWindows_48_144.png
Bar00_TopDownHouse_DoorsAndWindows_64_160.png
Bar00_TopDownHouse_DoorsAndWindows_64_176.png
Bar00_TopDownHouse_DoorsAndWindows_48_192.png
Bar00_TopDownHouse_DoorsAndWindows_48_208.png
Bar00_TopDownHouse_DoorsAndWindows_48_224.png
Bar00_TopDownHouse_DoorsAndWindows_48_240.png
Bar02_TopDownHouse_FloorsAndWalls_16_0.png
Bar02_TopDownHouse_FloorsAndWalls_16_272.png
Bar04_TopDownHouse_FurnitureState2_0_0.png
Bar04_TopDownHouse_FurnitureState2_0_16.png
Bar04_TopDownHouse_FurnitureState2_0_32.png
Bar04_TopDownHouse_FurnitureState2_0_48.png
Bar04_TopDownHouse_FurnitureState2_0_64.png
Bar04_TopDownHouse_FurnitureState2_0_80.png
Bar04_TopDownHouse_FurnitureState2_0_96.png
Bar04_TopDownHouse_FurnitureState2_0_112.png
Bar04_TopDownHouse_FurnitureState2_0_128.png
Bar04_TopDownHouse_FurnitureState2_0_144.png
Bar04_TopDo

---

In [14]:
import json
import numpy as np
import pandas as pd
import pickle

# load map tile layers, combine them and convert them to a movement map
def convert_map_to_movement_map(map_path:str, 
                                movement_dict_path:str='assets/maps/movespeed_dict.pkl') -> pd.DataFrame:
    
    with open(movement_dict_path, 'rb') as file:
        movement_dict = pickle.load(file)

    df_movement_map = pd.DataFrame()

    map_layers = load_map_layers(map_path)

    for map in map_layers:
        df_tmp = map.applymap(get_tile_id).replace(movement_dict)
        df_movement_map = df_movement_map.combine(df_tmp, np.minimum)
    
    return df_movement_map

def load_map_layers(map_path:str) -> list:

    with open(map_path) as file:
        data = json.load(file)
    
    map_layers = []
    for i in range(len(data['layers'])):
        map_layers.append(pd.DataFrame(np.reshape(data['layers'][i]['data'], (data['layers'][i]['height'], data['layers'][i]['width']))).sub(1))

    return map_layers

def get_tile_id(tile_id:int) -> int:
    if tile_id < 0:
        return tile_id
    return tile_id & 0b00001111111111111111111111111111

# There is no spin. It can be expressed as a combination of horizontal, vertical, diagonal flips.
def is_hoz_flip(tile_id:int) -> bool:
    return ((tile_id & 0b10000000000000000000000000000000) != 0) and (tile_id >= 0)

def is_ver_flip(tile_id:int) -> bool:
    return ((tile_id & 0b01000000000000000000000000000000) != 0) and (tile_id >= 0)

def is_dia_flip(tile_id:int) -> bool:
    return ((tile_id & 0b00100000000000000000000000000000) != 0) and (tile_id >= 0)

def is_hex_120_deg_flipped(tile_id:int) -> bool:
    return ((tile_id & 0b00010000000000000000000000000000) != 0) and (tile_id >= 0)

In [ ]:
res = convert_map_to_movement_map('assets/maps/test_cave/first_cave.json')

In [ ]:
pd.set_option('display.max_columns', None)

In [12]:
load_map_layers('graphics/verl.json')[1].applymap(get_tile_id)

/tmp/ipykernel_25352/1488826632.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  load_map_layers('graphics/verl.json')[1].applymap(get_tile_id)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,-1,421,422,421,421,422,423,424,433,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,-1,-1,-1,-1,-1,-1,-1,-1,529,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [17]:
id = 3758097060
print(is_hoz_flip(id), is_ver_flip(id), is_dia_flip(id), is_hex_120_deg_flipped(id))

True True True False


In [18]:
get_tile_id(id)

676